# GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers

En el paper [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323) se expone la necesidad de crear un método de cuantización post entrenamiento que no degrade la calidad del modelo. En este post hemos visto el método [llm.int8()](https://maximofn.com/llm-int8/) que cuantiza a INT8 algunos vectores de las matrices de pesos, siempre y cuando ninguno de sus valores sobrepase un valor umbral, lo cual está muy bien, pero no cuantizan todos los pesos del modelo. En este paper proponen un método que cuantiza todos los pesos del modelo a 4 y 3 bits, sin degradar la calidad del modelo. Lo que supone un ahorro considerable de memoria, no solo porque se cuantizan todos los pesos, sino porque además se hace a 4, 3 bits (y hasta 1 y 2 bits en ciertas condiciones), en vez de a 8 bits.

## Trabajos en los que se basa

### Cuantización por capas

Por un lado se basan en los trabajos `Nagel et al., 2020`; `Wang et al., 2020`; `Hubara et al., 2021` y `Frantar et al., 2022`, que proponen cuantizar los pesos de las capas de una red neuronal a 4 y 3 bits, sin degradar la calidad del modelo.

Teniendo un conjunto de datos `m` proveniente de un dataset, a cada capa `l` se le meten los datos y se obtiene la salida de los pesos `W` de dicha capa. Así que lo que se hace es buscar unos pesos nuevos `Ŵ` cuantizados que minimicen el error cuadrático en relación con la salida de la capa de precisión total

`argmin_Ŵ||WX− ŴX||^2`

Los valores de `Ŵ` se establecen antes de realizar el proceso de cuantización y durante el proceso, cada parámetro de `Ŵ` puede cambiar de valor independientemente sin depender del valor de los demás parámetros de `Ŵ`.

### Optimal brain quantization (OBQ)

En el trabajo de `OBQ` de `Frantar et al., 2022` optimizan el proceso de cuantización por capas anterior, haciendo que llegue a ser hasta 3 veces más rápido. Esto ayuda con los modelos grandes, ya que cuantizar un modelo grande puede llevar mucho tiempo.

El método `OBQ` es un enfoque para resolver el problema de cuantificación por capas en modelos de lenguaje. `OBQ` parte de la idea de que el error cuadrático se puede descomponer en la suma de errores individuales para cada fila de la matriz de pesos. Luego, el método cuantifica cada peso de manera independiente, actualizando siempre los pesos no cuantificados para compensar el error incurrido por la cuantificación.

El método es capaz de cuantificar modelos de tamaño mediano en tiempos razonables, pero como es un algoritmo de complejidad cúbica hace que sea extremadamente costoso aplicarlo a modelos con miles de millones de parámetros.

## Algoritmo de GPTQ

### Paso 1: Información de orden arbitrario

En `OBQ` se buscaba la fila de pesos que creaba menor error cuadrático medio para cuantizar, pero se dieron cuenta que al hacerlo de manera aleatoria no aumentaba mucho el error cuadrático medio final. Por lo que en vez de buscar la fila que minimiza el error cuadrático medio, que creaba una complejidad cúbica en el algoritmo, se hace siempre en el mismo orden. Gracias a esto se reduce mucho el tiempo de ejecución del algoritmo de cuantización.

### Paso 2: Actualizaciones lazy batch

Como la actualización de los pesos se hace fila a fila, hace que sea un proceso lento y no se aproveche del todo el hardware. Por lo que proponen realizar las actualizaciones en lotes de `B=128` filas. De esta manera se aprovecha mejor el hardware y se reduce el tiempo de ejecución del algoritmo.

### Paso 3: Reformulación de Cholesky

El problema de hacer las actualizaciones por lotes es que, debido a la gran escala de los modelos, se pueden producir errores numéricos que afectan la precisión del algoritmo. Concretamente, se pueden obtener matrices indefinidas, lo que provoca que el algoritmo actualice los pesos restantes en direcciones incorrectas, lo que resulta en una cuantización muy mala.

Para solucionar esto, los autores del paper proponen utilizar una reformulación de Cholesky, que es un método más numéricamente estable.

## Resultados de GPTQ

A continuación se muestran dos gráficas con la medida de la perplejidad (perplexity) en el dataset `WikiText2` para todos los tamaños de los modelos OPT y BLOOM. Se puede ver que con la técnica de cuantización RTN, la perplejidad en algunos tamaños aumenta mucho, mientras que con GPTQ se mantiene similar a la que se obtinene con el modelo en FP16

![GPTQ-figure1](https://maximofn.com/wp-content/uploads/2024/07/GPTQ-figure1.webp)

A continuación se muestran otras gráficas, pero con la medida de del accuracy en el dataset `LAMBADA`. Ocurre lo mismo, mientras que GPTQ se mantiene similar a lo obtenido con FP16, otros métodos de cuantización degradan mucho la calidad del modelo

![GPTQ-figure3](https://maximofn.com/wp-content/uploads/2024/07/GPTQ-figure3.webp)

## Cuantización extrema

En las gráficas anteriores se han mostrado los resultados de cuantizar el modelo a 3 y 4 bits, pero podemos cuantizarlos a 2bits, e incluso a 1 solo bit.

Modificando el tamaño de los batches al utilizar el algoritmo podemos obtener buenos resultados cuantizando tanto el modelo

|Modelo | FP16 | g128 | g64 | g32 | 3 bits|
|-------|-------|--------|--------|--------|---------|
|OPT-175B | 8.34 | 9.58 | 9.18 | 8.94 | 8.68|
|BLOOM | 8.11 | 9.55 | 9.17 | 8.83 | 8.64|

En la tabla anterior se puede ver el resultado de la perplejidad en el dataset `WikiText2` para los modelos `OPT-175B` y `BLOOM` cuantizados a 3 bits. Se puede ver que a medida que se usen batches más pequeños, la perplejidad disminuye, lo que significa que la calidad del modelo cuantizado es mejor. Pero tiene el problema de que el algoritmo tarda más en ejecutarse

## Descuantificación dinámica en la inferencia

Durante la inferencia se raliza algo llamado `descuantificación dinámica` (`dynamic dequantization`) para poder realizar la inferencia. Se va descuantificando cada capa a medida que se va pasando por ellas.

Para ello desarrollaron un kernel de que descuantifica las matrices y realiza los productos matriciales. Si bien la descuantificación consume más cálculos, el kernel tiene que acceder a mucha menos memoria, lo que genera aceleraciones significativas

La inferencia se realiza en FP16 descuantizando los pesos a medida que se va pasando por las capas y la función de activación de cada capa también se realiza en FP16. Aunque esto hace que haya que hacer más cálculos, porque hay que descuantizar, esos cálculos hacen que el proceso total sea más rápido, porque hay que traer de la memoria menos datos. Hay que traer de la memoria los pesos en menos bits, por lo que al final, en matrices de muchos parámetros hace que se ahorren muchos datos. El cuello de botella normalmente está en traer los datos de la memoria, por lo que aunque haya que hacer más cálculos, al final la inferencia es más rápida

## Velocidad de inferencia

Los autores del paper realizaron una prueba cuantizando el modelo BLOOM-175B a 3 bits, lo que ocupaba unos 63 GB de memoria VRAM, incluidos los embeddings y la capa de salida que se mantienen en FP16. Además mantener la ventana de contexto de 2048 tokens consume unos 9 GB de memoria, lo que hace en total unos 72 GB de memoria VRAM. Cuantizaron en 3 bits y no en 4 para poder realizar este experimento y poder meter el modelo en una sola GPU Nvidia A100 de 80 GB de memoria VRAM.

Para comparar, la inferencia normal en FP16 requiere unos 350 GB de memoria VRAM, lo que equivale a 5 GPUs Nvidia A100 de 80 GB de memoria VRAM. Y la inferencia cuantizando en 8 bits mediante [llm.int8()](https://maximofn.com/llm-int8/) requiere 3 de dichas GPUs.

A continuación, se muestra una tabla con la inferencia del modelo en FP16 y cuantizado a 3 bits e GPUs Nvidia A100 de 80 GB de memoria VRAM y Nvidia A6000 de 48 GB de memoria VRAM.

|GPU (VRAM)| tiempo promedio por token en FP16 (ms) | tiempo promedio por token en 3 bit (ms) | Aceleración | Reducción de GPUs necesarias |
|-------|---------|--------|-------------|------------------|
|A6000 (48GB) | 589 | 130 | ×4.53 | 8→ 2|
|A100 (80GB) | 230 | 71 | ×3.24 | 5→ 1|

Por ejemplo, utilizando los kernels, el modelo OPT-175B de 3 bits se ejecuta en una sola A100 (en vez de en 5) y es aproximadamente 3,25 veces más rápido que la versión FP16 en términos de tiempo promedio por token.

La GPU NVIDIA A6000 tiene un ancho de banda de memoria mucho menor, por lo que esta estrategia es aún más efectiva: ejecutar el modelo OPT-175B de 3 bits en 2 GPUs A6000 (en vez de en 8) es aproximádamente 4.53 veces más rápido que la versión FP16.

## Librerías

Los autores del paper implementaron la librería [GPTQ](https://github.com/IST-DASLab/gptq). Otras librerías fueron creadas como [GPTQ-for-LLaMa](https://github.com/qwopqwop200/GPTQ-for-LLaMa), [exllama](https://github.com/turboderp/exllama) y [llama.cpp](https://github.com/ggerganov/llama.cpp/). Sin embargo estas librerías se centran solo en la arquitectura llama, por lo que la librería [AutoGPTQ](https://github.com/AutoGPTQ/AutoGPTQ) fue la que ganó más popularidad porque tiene una cobertura más amplia de arquitecturas.

Por ello dicha librería [AutoGPTQ](https://github.com/AutoGPTQ/AutoGPTQ) se integró mediante una API dentro de la librería [transformers](https://maximofn.com/hugging-face-transformers/). Para poder usarla es necesario instalarla como se indica en la seción [Installation](https://github.com/AutoGPTQ/AutoGPTQ#installation) de su repositorio y tener instalada la librería [optimun](https://maximofn.com/hugging-face-optimun/).

## Cuantización de un modelo

Vamos a ver cómo cuantizar un modelo con la librería [optimun](https://maximofn.com/hugging-face-optimun/) y la API de [AutoGPTQ](https://github.com/AutoGPTQ/AutoGPTQ).

In [ ]:
# !pip install -q -U transformers peft accelerate optimum dataset

In [ ]:
# !pip install auto-gptq --no-build-isolation --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

In [ ]:
# !pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

### Inferencia del modelo no cuantizado

Vamos a cuantizar el modelo [facebook/opt-125m](https://huggingface.co/facebook/opt-125m) que como su nombre indica es un modelo de 125M de parámetros, por lo que en FP16 necesitaríamos 0.25 GB de memoria VRAM. Primero ejecutamos el modelo para ver la memoria que ocupa y la salida que genera

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = "facebook/opt-125m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).half().to(device)

In [3]:
model_memory = model.get_memory_footprint()/(1024**3)
print(f"Model memory: {model_memory:.2f} GB")

Model memory: 0.23 GB


In [5]:
import time

input_tokens = tokenizer("Hello my name is Maximo and I am a Machine Learning Engineer", return_tensors="pt").to(model.device)

t0 = time.time()
max_new_tokens = 50
outputs = model.generate(
    input_ids=input_tokens.input_ids,
    attention_mask=input_tokens.attention_mask,
    max_length=input_tokens.input_ids.shape[1] + max_new_tokens,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Inference time: {time.time() - t0:.2f} s")

Hello my name is Maximo and I am a Machine Learning Engineer. I am currently working on a project that will allow me to learn Machine Learning in a way that is not only practical but also fun. I am currently working on a project that will allow me to learn Machine Learning in a way that is not only
Inference time: 1.30 s


### Cuantización del modelo a 4 bits

Vamos a cuantizarlo a 4 bits. Reinico el notebook para no tener problemas de memoria

Primero creo el tokenizador

In [1]:
from transformers import AutoTokenizer

checkpoint = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Ahora creamos la configuración de cuantización. Como hemos dicho, este algoritmo calcula el error de los pesos cuantizados sobre los originales en función de entradas de un dataset, por lo que en la configuración tenemos que pasarle con qué dataset queremos cuantizar el modelo.

Los disponibles por defecto son `wikitext2`,`c4`,`c4-new`,`ptb` y `ptb-new`.

También podemos crear nosotros un dataset a partir de una lista de strings

```python
dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]
```

Además le tenemos que decir el número de bits que tenga el modelo cuantizado mediante el parámetro `bits`

In [2]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)

In [6]:
from transformers import AutoModelForCausalLM
import time

t0 = time.time()
model_4bits = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=quantization_config)
t_quantization = time.time() - t0
print(f"Quantization time: {t_quantization:.2f} s = {t_quantization/60:.2f} min")

Quantization time: 1952.48 s = 32.54 min


Vamos a ver la memoria que ocupa ahora

In [4]:
model_4bits_memory = model_4bits.get_memory_footprint()/(1024**3)
print(f"Model memory: {model_4bits_memory:.2f} GB")

Model memory: 0.12 GB


Hacemos la inferencia y vemos el tiempo que tarda

In [9]:
import time

input_tokens = tokenizer("Hello my name is Maximo and I am a Machine Learning Engineer", return_tensors="pt").to(model_4bits.device)

t0 = time.time()
max_new_tokens = 50
outputs = model_4bits.generate(
    input_ids=input_tokens.input_ids,
    attention_mask=input_tokens.attention_mask,
    max_length=input_tokens.input_ids.shape[1] + max_new_tokens,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Inference time: {time.time() - t0:.2f} s")

Hello my name is Maximo and I am a Machine Learning Engineer. I am a graduate of the University of California, Berkeley. I am a member of the Computer Science and Engineering Department at the University of California, Berkeley. I am a member of the Computer Science and Engineering Department at the University of California, Berkeley
Inference time: 1.91 s


### Cuantización del modelo a 3 bits

Vamos a cuantizarlo a 3 bits. Reinico el notebook para no tener problemas de memoria

Primero creo el tokenizador

In [6]:
from transformers import AutoTokenizer

checkpoint = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Creamos la configuración de cuantización

In [7]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(bits=3, dataset = "c4", tokenizer=tokenizer)

In [13]:
from transformers import AutoModelForCausalLM
import time

t0 = time.time()
model_3bits = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=quantization_config)
t_quantization = time.time() - t0
print(f"Quantization time: {t_quantization:.2f} s = {t_quantization/60:.2f} min")

Quantization time: 1941.64 s = 32.36 min


Vamos a ver la memoria que ocupa ahora

In [9]:
model_3bits_memory = model_3bits.get_memory_footprint()/(1024**3)
print(f"Model memory: {model_3bits_memory:.2f} GB")

Model memory: 0.11 GB


Hacemos la inferencia y vemos el tiempo que tarda

In [11]:
import time

input_tokens = tokenizer("Hello my name is Maximo and I am a Machine Learning Engineer", return_tensors="pt").to(model_3bits.device)

t0 = time.time()
max_new_tokens = 50
outputs = model_3bits.generate(
    input_ids=input_tokens.input_ids,
    attention_mask=input_tokens.attention_mask,
    max_length=input_tokens.input_ids.shape[1] + max_new_tokens,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Inference time: {time.time() - t0:.2f} s")

Hello my name is Maximo and I am a Machine Learning Engineer. I am currently working on a new project that I am working on. I am currently working on a new project that I am working on. I am currently working on a new project that I am working on. I am currently working on a new
Inference time: 4.45 s


Guardamos el modelo

In [14]:
save_folder = "./model_3bits/"
model_3bits.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

('./model_3bits/tokenizer_config.json',
 './model_3bits/special_tokens_map.json',
 './model_3bits/vocab.json',
 './model_3bits/merges.txt',
 './model_3bits/added_tokens.json',
 './model_3bits/tokenizer.json')

Creamos la model card y lo subimos al hub

In [ ]:
from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
repo_id = "opt-125m-GPTQ-3bits"
commit_message = f"AutoGPTQ model for {checkpoint}: {quantization_config.bits}bits, gr{quantization_config.group_size}, desc_act={quantization_config.desc_act}"
model_3bits.push_to_hub(repo_id, commit_message=commit_message)

### Cuantización del modelo a 2 bits

Vamos a cuantizarlo a 2 bits. Reinico el notebook para no tener problemas de memoria

Primero creo el tokenizador

In [1]:
from transformers import AutoTokenizer

checkpoint = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Creamos la configuración de cuantización

In [2]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(bits=2, dataset = "c4", tokenizer=tokenizer, group_size=2)

In [3]:
from transformers import AutoModelForCausalLM
import time

t0 = time.time()
model_2bits = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=quantization_config)
t_quantization = time.time() - t0
print(f"Quantization time: {t_quantization:.2f} s = {t_quantization/60:.2f} min")

CUDA extension not installed.
CUDA extension not installed.


Quantizing model.decoder.layers blocks :   0%|          | 0/12 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/home/sae00531/miniconda3/envs/nlp_/lib/python3.11/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


Quantization time: 1842.62 s = 30.71 min


Vamos a ver la memoria que ocupa ahora

In [4]:
model_2bits_memory = model_2bits.get_memory_footprint()/(1024**3)
print(f"Model memory: {model_2bits_memory:.2f} GB")

Model memory: 0.18 GB


Hacemos la inferencia y vemos el tiempo que tarda

In [5]:
import time

input_tokens = tokenizer("Hello my name is Maximo and I am a Machine Learning Engineer", return_tensors="pt").to(model_2bits.device)

t0 = time.time()
max_new_tokens = 50
outputs = model_2bits.generate(
    input_ids=input_tokens.input_ids,
    attention_mask=input_tokens.attention_mask,
    max_length=input_tokens.input_ids.shape[1] + max_new_tokens,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Inference time: {time.time() - t0:.2f} s")

Hello my name is Maximo and I am a Machine Learning Engineer. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work.
Inference time: 2.27 s


Guardamos el modelo

In [6]:
save_folder = "./model_2bits/"
model_2bits.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

('./model_2bits/tokenizer_config.json',
 './model_2bits/special_tokens_map.json',
 './model_2bits/vocab.json',
 './model_2bits/merges.txt',
 './model_2bits/added_tokens.json',
 './model_2bits/tokenizer.json')

Creamos la model card y lo subimos al hub

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_id = "opt-125m-GPTQ-2bits"
commit_message = f"AutoGPTQ model for {checkpoint}: {quantization_config.bits}bits, gr{quantization_config.group_size}, desc_act={quantization_config.desc_act}"
model_2bits.push_to_hub(repo_id, commit_message=commit_message)

model.safetensors:   0%|          | 0.00/198M [00:00<?, ?B/s]

RuntimeError: Error while uploading 'model.safetensors' to the Hub.

model.safetensors:   0%|          | 0.00/198M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Maximofn/opt-125m-GPTQ-2bits/commit/94a1bd81527840495391d4d50a0e66ad7554e2de', commit_message='AutoGPTQ model for facebook/opt-125m: 2bits, gr2, desc_act=False', commit_description='', oid='94a1bd81527840495391d4d50a0e66ad7554e2de', pr_url=None, pr_revision=None, pr_num=None)

### Cuantización del modelo a 1 bit

Vamos a cuantizarlo a 1 bit. Reinico el notebook para no tener problemas de memoria

Primero creo el tokenizador

In [ ]:
from transformers import AutoTokenizer

checkpoint = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Creamos la configuración de cuantización

In [ ]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(bits=2, dataset = "c4", tokenizer=tokenizer, group_size=2)

In [ ]:
from transformers import AutoModelForCausalLM
import time

t0 = time.time()
model_2bits = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=quantization_config)
t_quantization = time.time() - t0
print(f"Quantization time: {t_quantization:.2f} s = {t_quantization/60:.2f} min")

CUDA extension not installed.
CUDA extension not installed.


Quantizing model.decoder.layers blocks :   0%|          | 0/12 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

/home/sae00531/miniconda3/envs/nlp_/lib/python3.11/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


Quantization time: 1829.43 s = 30.49 min


Vamos a ver la memoria que ocupa ahora

In [ ]:
model_2bits_memory = model_2bits.get_memory_footprint()/(1024**3)
print(f"Model memory: {model_2bits_memory:.2f} GB")

Model memory: 0.18 GB


Hacemos la inferencia y vemos el tiempo que tarda

In [ ]:
import time

input_tokens = tokenizer("Hello my name is Maximo and I am a Machine Learning Engineer", return_tensors="pt").to(model_2bits.device)

t0 = time.time()
max_new_tokens = 50
outputs = model_2bits.generate(
    input_ids=input_tokens.input_ids,
    attention_mask=input_tokens.attention_mask,
    max_length=input_tokens.input_ids.shape[1] + max_new_tokens,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Inference time: {time.time() - t0:.2f} s")

Hello my name is Maximo and I am a Machine Learning Engineer. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work. I have a lot of work.
Inference time: 1.97 s


Guardamos el modelo

In [ ]:
save_folder = "./model_2bits/"
model_2bits.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

('./model_2bits/tokenizer_config.json',
 './model_2bits/special_tokens_map.json',
 './model_2bits/vocab.json',
 './model_2bits/merges.txt',
 './model_2bits/added_tokens.json',
 './model_2bits/tokenizer.json')

Creamos la model card y lo subimos al hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_id = "opt-125m-GPTQ-2bits"
commit_message = f"AutoGPTQ model for {checkpoint}: {quantization_config.bits}bits, gr{quantization_config.group_size}, desc_act={quantization_config.desc_act}"
model_2bits.push_to_hub(repo_id, commit_message=commit_message)

model.safetensors:   0%|          | 0.00/198M [00:00<?, ?B/s]